In [9]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
from aws_setup import *

In [3]:
name='fast-ai'

In [4]:
instance = create_spot_instance(name); instance

Waiting on spot fullfillment...
Fullfillment completed. InstanceId: i-0611c3faf48066b6b
Rebooting...


ec2.Instance(id='i-0611c3faf48066b6b')

In [5]:
instance.public_ip_address

'35.167.196.233'

In [6]:
def attach_volume(instance, volume_tag, device='/dev/xvdf'):
    volumes = list(ec2.volumes.filter(Filters=[{'Name': 'tag-value', 'Values': [volume_tag]}]))
    if not volumes: print('Could not find volume for tag:', volume_tag); return
    instance.attach_volume(Device=device, VolumeId=volumes[0].id)
    instance.reboot()
    instance.wait_until_running()
    return instance
    
# TODO: need to make sure ebs is formatted correctly

In [8]:
attach_volume(instance, f'{name}-ebs-volume-test')

ec2.Instance(id='i-0611c3faf48066b6b')

In [11]:
get_ssh_command(instance)

'ssh -i ~/.ssh/aws-key-fast-ai.pem ubuntu@35.167.196.233'

In [ ]:
instance.wait_until_running

In [ ]:
instance.reboot()

In [ ]:
def create_volume(name, size=120, volume_type='gp2'):
    tag_specs = [{
        'Tags': [{
            'Key': 'Name',
            'Value': f'{name}-ebs-volume'
        }]
    }]
    volume = ec2.create_volume(Size=size, VolumeType=volume_type, TagSpecifications=tag_specs)
    

### Creating and attaching EFS

In [14]:
efsc = session.client('efs')

In [ ]:
get_vpc_ids()

In [ ]:
def create_efs(name):
    vpc_id, sg_id, subnet_id = get_vpc_ids(name)
    efsc = session.client('efs')
    efs_response = efsc.create_file_system(CreationToken=f'{name}-efs', PerformanceMode='generalPurpose')
    efs_id = efs_response['FileSystemId']
    efsc.create_tags(FileSystemId=efs_id, Tags=[{'Key': 'Name', 'Value': f'{name}-efs'}])
    
    mount_target = efsc.create_mount_target(FileSystemId=efs_id,
                                              SubnetId=subnet_id,
                                              SecurityGroups=[sg_id])
    return mount_target

In [16]:
file_system = efsc.create_file_system(CreationToken=f'{name}-efs', PerformanceMode='generalPurpose')

In [27]:
efs_id = file_system['FileSystemId']


In [20]:
efsc.create_tags(FileSystemId=efs_id, Tags=[{'Key': 'Name', 'Value': f'{name}-efs'}])

{'ResponseMetadata': {'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Thu, 29 Mar 2018 05:50:50 GMT',
   'x-amzn-requestid': '232ef638-3315-11e8-8bc6-65160ec0c332'},
  'HTTPStatusCode': 204,
  'RequestId': '232ef638-3315-11e8-8bc6-65160ec0c332',
  'RetryAttempts': 0}}

In [22]:
efsc.describe_mount_targets(FileSystemId=efs_id)

{'MountTargets': [],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '51',
   'content-type': 'application/json',
   'date': 'Thu, 29 Mar 2018 05:52:01 GMT',
   'x-amzn-requestid': '4d31f792-3315-11e8-bac7-050c3ff218d4'},
  'HTTPStatusCode': 200,
  'RequestId': '4d31f792-3315-11e8-bac7-050c3ff218d4',
  'RetryAttempts': 0}}

In [24]:
vpc_id, sg_id, subnet_id = get_vpc_ids(name)

In [28]:

response = efsc.create_mount_target(FileSystemId=efs_id,
                                          SubnetId=subnet_id,
                                          SecurityGroups=[sg_id])

In [29]:
response

{'FileSystemId': 'fs-0ea233a7',
 'IpAddress': '10.0.0.6',
 'LifeCycleState': 'creating',
 'MountTargetId': 'fsmt-e4148a4d',
 'NetworkInterfaceId': 'eni-c2c9a4e5',
 'OwnerId': '698042698196',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '203',
   'content-type': 'application/json',
   'date': 'Thu, 29 Mar 2018 05:54:26 GMT',
   'x-amzn-requestid': 'a30825cd-3315-11e8-bac7-050c3ff218d4'},
  'HTTPStatusCode': 200,
  'RequestId': 'a30825cd-3315-11e8-bac7-050c3ff218d4',
  'RetryAttempts': 0},
 'SubnetId': 'subnet-f056ff89'}

In [32]:
efsc.describe_mount_targets(FileSystemId=efs_id)

{'MountTargets': [{'FileSystemId': 'fs-0ea233a7',
   'IpAddress': '10.0.0.6',
   'LifeCycleState': 'available',
   'MountTargetId': 'fsmt-e4148a4d',
   'NetworkInterfaceId': 'eni-c2c9a4e5',
   'OwnerId': '698042698196',
   'SubnetId': 'subnet-f056ff89'}],
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '255',
   'content-type': 'application/json',
   'date': 'Thu, 29 Mar 2018 06:00:06 GMT',
   'x-amzn-requestid': '6e918eab-3316-11e8-b37c-e96e22745ef1'},
  'HTTPStatusCode': 200,
  'RequestId': '6e918eab-3316-11e8-b37c-e96e22745ef1',
  'RetryAttempts': 0}}

In [ ]:
response = efs_client.describe_file_systems()